# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.09570


extracting tarball to tmp_2303.09570...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.09570/clearsurvey.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'defs' from 'tmp_2303.09570/defs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2303.09836...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.09845


extracting tarball to tmp_2303.09845... done.
Retrieving document from  https://arxiv.org/e-print/2303.09876


extracting tarball to tmp_2303.09876...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09637-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09637) | **Origin of the diffuse 4-8 keV emission in M82**  |
|| K. Iwasawa, et al. -- incl., <mark>M. A. Perez-Torres</mark> |
|*Appeared on*| *2023-03-20*|
|*Comments*| *19 pages, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| We present the first spatially resolved, X-ray spectroscopic study of the 4-8 keV diffuse emission found in the central part of the nearby starburst galaxy M82 on a few arcsecond scales. The new details that we see allow a number of important conclusions to be drawn on the nature of the hot gas and its origin as well as feedback on the ISM. We use archival data from Chandra with an exposure time of 570 ks. The Fexxv emission at 6.7 keV, expected from metal-enriched hot gas, is enhanced only in a limited area close to the starburst disc and is weak or almost absent over the rest of the diffuse emission, resulting in spatial variations in EW from <0.1 keV to 1.9 keV. This shows the presence of non-thermal emission due to inverse Compton scattering of the FIR photons by radio emitting cosmic ray electrons. The morphological resemblance between the diffuse X-ray, radio, and FIR emission maps supports this concept. Our decomposition of the diffuse emission spectrum indicates that ~70% of the 4-8 keV luminosity originates from the inverse Compton emission. The metal-rich hot gas with kT~5 keV makes a minor contribution to the 4-8 keV continuum, but it accounts for the majority of the observed Fexxv line. This hot gas appears to emerge from the circumnuclear starburst ring and fill the galactic chimneys identified as mid-infrared and radio emission voids. The energetics argument suggests that much of the supernova energy in the starburst site has gone into creating the chimneys and is transported to the halo. We argue that a hot, rarefied environment produced by strong supernova feedback results in displacing the brightest X-ray and radio supernova remnants which are instead found to reside in GMC. We find a faint X-ray source with a radio counterpart, close to the kinematic centre of the galaxy and we carefully examine the possibility that this source is a low-luminosity AGN in ADAF phase. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09710-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09710) | **The Analyses of Globular Clusters Pulsars and Their Detection Efficiency**  |
|| De-Jiang Yin, et al. -- incl., <mark>Li-Yun Zhang</mark>, <mark>Bao-Da Li</mark>, <mark>Ming-Hui Li</mark> |
|*Appeared on*| *2023-03-20*|
|*Comments*| *9 pages,5 figures, accepted by RAA, comments are always welcomed!*|
|**Abstract**| Up to November 2022, 267 pulsars have been discovered in 36 globular clusters (GCs). In this paper, we present our studies on the distribution of GC pulsar parameters and the detection efficiency. The power law relation between the average of dispersion measure ($\overline{\rm DM}$) and dispersion measure difference ($\Delta {\rm DM}$) of known pulsars in GCs is $\lg\Delta {\rm DM} \propto 1.52 \lg \overline{\rm DM}$. The sensitivity could be the key to find more pulsars. As a result, several years after the construction of a radio telescope, the GC pulsar number will be increased accordingly. We suggest that currently GCs in the southern hemisphere could have higher possibilities to find new pulsars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09714-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09714) | **Periodic X-ray sources in the Massive Globular Cluster 47 Tucanae:  Evidence for Dynamically Formed Cataclysmic Variables**  |
|| Tong Bao, <mark>Zhiyuan Li</mark>, Zhongqun Cheng |
|*Appeared on*| *2023-03-20*|
|*Comments*| *20 pages, 10 figures, Accepted for publication in MNRAS*|
|**Abstract**| We present a systematic study of periodic X-ray sources in the massive globular cluster 47 Tuc, utilizing deep archival Chandra observations that resolve the cluster core and recently available eROSITA observations that cover the cluster outskirt. By applying the Gregory-Loredo algorithm, we detect 20 periodic signals among 18 X-ray sources, ranging between 205-95731 second. Fourteen periods are newly discovered in the X-ray band. We classify these periodic sources into four quiescent low-mass X-ray binaries, one milli-second pulsar, two coronally-active binaries and eleven cataclysmic variables (CVs), based on their X-ray temporal and spectral properties, as well as multi-band information. Despite a small sample subject to potential selection bias against faint and non-magnetic CVs, the 11 CVs together define an orbital period distribution significantly different from that of the CVs previously found in the solar neighborhood and the Galactic bulge. In particular, there exists in 47 Tuc an apparent paucity of short-period CVs below the period gap, which might be attributed to a high occupation fraction of non-magnetic CVs. Also characteristic of the 47 Tuc CVs are an overabundance of long-period CVs with a subgiant donor, a substantial fraction of CVs within the period gap, and a steep radial surface density profile. These are best understood as a group of CVs having recently formed via dynamical interactions in the dense cluster core. Despite sufficient sensitivity of the X-ray data, only one periodic source is found between one-third of the half-light radius and the tidal radius, the nature of which is unclear. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09762-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09762) | **Aemulus $ν$: Precise Predictions for Matter and Biased Tracer Power  Spectra in the Presence of Neutrinos**  |
|| <mark>Joseph DeRose</mark>, et al. -- incl., <mark>Zhongxu Zhai</mark> |
|*Appeared on*| *2023-03-20*|
|*Comments*| *35 pages, 14 figures, to be submitted to JCAP*|
|**Abstract**| We present the Aemulus $\nu$ simulations: a suite of 150 $(1.05 h^{-1}\rm Gpc)^3$ $N$-body simulations with a mass resolution of $3.51\times 10^{10} \frac{\Omega_{cb}}{0.3} ~ h^{-1} M_{\odot}$ in a $w\nu$CDM cosmological parameter space. The simulations have been explicitly designed to span a broad range in $\sigma_8$ to facilitate investigations of tension between large scale structure and cosmic microwave background cosmological probes. Neutrinos are treated as a second particle species to ensure accuracy to $0.5\, \rm eV$, the maximum neutrino mass that we have simulated. By employing Zel'dovich control variates, we increase the effective volume of our simulations by factors of $10-10^5$ depending on the statistic in question. As a first application of these simulations, we build new hybrid effective field theory and matter power spectrum surrogate models, demonstrating that they achieve $\le 1\%$ accuracy for $k\le 1\, h\,\rm Mpc^{-1}$ and $0\le z \le 3$, and $\le 2\%$ accuracy for $k\le 4\, h\,\rm Mpc^{-1}$ for the matter power spectrum. We publicly release the trained surrogate models, and estimates of the surrogate model errors in the hope that they will be broadly applicable to a range of cosmological analyses for many years to come. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09836-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09836) | **An in-depth exploration of LAMOST Unknown spectra based on density  clustering**  |
|| Haifeng Yang, et al. -- incl., <mark>Xujun Zhao</mark> |
|*Appeared on*| *2023-03-20*|
|*Comments*| *18 pages, 15 figures*|
|**Abstract**| LAMOST (Large Sky Area Multi-Object Fiber Spectroscopic Telescope) has completed the observation of nearly 20 million celestial objects, including a class of spectra labeled `Unknown'. Besides low signal-to-noise ratio, these spectra often show some anomalous features that do not work well with current templates. In this paper, a total of 638,000 `Unknown' spectra from LAMOST DR5 are selected, and an unsupervised-based analytical framework of `Unknown' spectra named SA-Frame (Spectra Analysis-Frame) is provided to explore their origins from different perspectives. The SA-Frame is composed of three parts: NAPC-Spec clustering, characterization and origin analysis. First, NAPC-Spec(Nonparametric density clustering algorithm for spectra) characterizes different features in the "unknown" spectrum by adjusting the influence space and divergence distance to minimize the effects of noise and high dimensionality, resulting in 13 types. Second, characteristic extraction and representation of clustering results are carried out based on spectral lines and continuum, where these 13 types are characterized as regular spectra with low S/Ns, splicing problems, suspected galactic emission signals, contamination from city light and un-gregarious type respectively. Third, a preliminary analysis of their origins is made from the characteristics of the observational targets, contamination from the sky, and the working status of the instruments. These results would be valuable for improving the overall data quality of large-scale spectral surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09845-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09845) | **The `blazar sequence' in TeV band**  |
|| Zhihao Ouyang, et al. -- incl., <mark>Zelin Li</mark> |
|*Appeared on*| *2023-03-20*|
|*Comments*| *19 pages, 4 figures and 4 tables. Accepted to APJ*|
|**Abstract**| The `blazar sequence' has been proposed for more than 20 years, yet its nature is still unclear. In this work, for the first time, we expand this topic to the TeV band by using a sample of 58 TeV blazars including 48 blazars in the quiescent state and 21 blazars in the flaring state. We investigate the correlation between the TeV luminosity, which has been compensated for attenuation from extragalactic background light, and the synchrotron peak frequency. We note that there is no correlation between TeV luminosity and peak frequency in the quiescent state and a strong anti-correlation in the flaring state for the observed value. However, there is a strong positive correlation in both the quiescent state and the flaring state for the intrinsic value. This indicates that the blazar sequence is shown in the flaring state rather than in the quiescent state for the observed value and the blazar sequence is not present in both two states after removing the beaming effect. In addition, to confirm whether the beaming effect results in the blazar sequence, we compare the \textit{Fermi} $\gamma$-ray luminosity between the quiescent state and the flaring state. We find the \textit{Fermi} $\gamma$-ray luminosity in the flaring state is greater than that in the quiescent state and the Doppler factor in the flaring state is greater. We suggest the blazar sequence in the flaring state may be due to the stronger beaming effect. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09876-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09876) | **Cosmology inference at the field level from biased tracers in  redshift-space**  |
|| <mark>Julia Stadler</mark>, Fabian Schmidt, Martin Reinecke |
|*Appeared on*| *2023-03-20*|
|*Comments*| **|
|**Abstract**| Cosmology inference of galaxy clustering at the field level with the EFT likelihood in principle allows for extracting all non-Gaussian information from quasi-linear scales, while robustly marginalizing over any astrophysical uncertainties. A pipeline in this spirit is implemented in the \texttt{LEFTfield} code, which we extend in this work to describe the clustering of galaxies in redshift space. Our main additions are: the computation of the velocity field in the LPT gravity model, the fully nonlinear displacement of the evolved, biased density field to redshift space, and a systematic expansion of velocity bias. We test the resulting analysis pipeline by applying it to synthetic data sets with a known ground truth at increasing complexity: mock data generated from the perturbative forward model itself, sub-sampled matter particles, and dark matter halos in N-body simulations. By fixing the initial-time density contrast to the ground truth, while varying the growth rate $f$, bias coefficients and noise amplitudes, we perform a stringent set of checks. These show that indeed a systematic higher-order expansion of the velocity bias is required to infer a growth rate consistent with the ground truth within errors. Applied to dark matter halos, our analysis yields unbiased constraints on $f$ at the level of a few percent for a variety of halo masses at redshifts $z=0,\,0.5,\,1$ and for a broad range of cutoff scales $0.08\,h/\mathrm{Mpc} \leq \Lambda \leq 0.20\,h/\mathrm{Mpc}$. Importantly, deviations between true and inferred growth rate exhibit the scaling with halo mass, redshift and cutoff that one expects based on the EFT of Large Scale Structure. Further, we obtain a robust detection of velocity bias through its effect on the redshift-space density field and are able to disentangle it from higher-derivative bias contributions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.09570-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.09570) | **CLEAR: Survey Overview, Data Analysis and Products**  |
|| Raymond C. Simons, et al. -- incl., <mark>Ivelina G. Momcheva</mark> |
|*Appeared on*| *2023-03-20*|
|*Comments*| *Accepted in ApJS. 29 pages, 14 figures. The data products (1D + 2D spectra, emission line maps) and catalogs (photometric and spectroscopic catalogs of GOODS-N and GOODS-S) are available at this https URL*|
|**Abstract**| We present an overview of the CANDELS Lyman-a Emission At Reionization (CLEAR) survey. CLEAR is a 130 orbit program of the Hubble Space Telescope using the Wide Field Camera 3 (WFC3) IR G102 grism. CLEAR targets 12 pointings divided between the GOODS-N and GOODS-S fields of the Cosmic Assembly Near-IR Deep Extragalactic Legacy Survey (CANDELS). Combined with existing spectroscopic data from other programs, the full CLEAR dataset includes spectroscopic imaging of these fields over 0.8-1.7 um. In this Paper, we describe the CLEAR survey, the survey strategy, the data acquisition, reduction, processing, and science products and catalogs released alongside this paper. The catalogs include emission line fluxes and redshifts derived from the combination of the photometry and grism spectroscopy for 6048 galaxies, primarily ranging from 0.2 < z < 3. We also provide an overview of CLEAR science goals and results. In conjunction with this Paper we provide links to electronic versions of the data products, including 1D + 2D extracted spectra and emission line maps. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure GS1_49742_Ha_compare.pdf</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

200  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6

  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
